#### Dajmila AZZOUZ / M2 probabilités et statistiques des nouvelles données

#### Estimation du Best Estimate avec Monte Carlo

In [ ]:
import numpy as np
# Définition des paramètres
r = 0.05  # taux sans risque court
x = 0.01  # spread entre rs et r
T = 10  # horizon de projection
h = 1/12  # pas de temps
M = 100000  # nombre de simulations
VR = 100  # valeur de rachat
mu = 0.01  # intensité de rachat

# Simulation des trajectoires de r(t) et x(t)
N = int(T/h)
t = np.linspace(0, T, N+1)
dB1 = np.sqrt(h)*np.random.normal(size=(M, N+1))
B1 = np.cumsum(dB1, axis=1)
r_t = r*np.ones((M, N+1)) + x*B1

dB2 = np.sqrt(h)*np.random.normal(size=(M, N+1))
B2 = np.cumsum(dB2, axis=1)
x_t = x*np.ones((M, N+1)) + 0.1*B2

# Calcul du Best Estimate par la méthode de Monte Carlo
S = np.zeros(M)
PM = np.zeros(M)
for i in range(M):
    Q = VR
    for j in range(1, N+1):
        L = np.exp(-mu*(t[j]-t[j-1])) # facteur de rachat
        Z = np.random.poisson(mu*L) # nombre de rachats pendant l'intervalle
        Q = Q*np.exp(-r_t[i,j]*(t[j]-t[j-1])) # actualisation du contrat
        if Z > 0:
            PM[i] = PM[i] + VR*np.exp(-r_t[i,j]*(t[j]-t[j-1]))*Z # provisionnement de marge en cas de rachat
            Q = VR # rachat total en cas de rachat
        S[i] = S[i] + Q

BE = np.mean(S) # Best Estimate
PM = np.mean(PM) # Provisionnement de Marge
# Calcul de l'intervalle de confiance à 95%
alpha = 0.05
m = np.mean(S)
s = np.std(S, ddof=1)
n = len(S)
t = np.abs(np.random.standard_t(df=n-1, size=100000))
t_alpha = np.percentile(t, 100*(1-alpha/2))
CI_low = m - t_alpha*s/np.sqrt(n)
CI_high = m + t_alpha*s/np.sqrt(n)

print("Best Estimate: {:.2f}".format(BE))
print("Le Provisionnement de Marge est :", round(PM, 2))
print("Intervalle de confiance à 95%: [{:.2f}, {:.2f}]".format(CI_low, CI_high))


In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))
for i in range(10):
    ax1.plot(t, r_t[i], label=f"Simulation {i+1}")
    ax2.plot(t, x_t[i], label=f"Simulation {i+1}")
ax1.set_xlabel("Temps")
ax1.set_ylabel("Taux d'intérêt")
ax1.set_title("Trajectoires de r(t)")
ax1.legend(bbox_to_anchor=(1.05, 1))

ax2.set_xlabel("Temps")
ax2.set_ylabel("Spread")
ax2.set_title("Trajectoires de x(t)")
ax2.legend(bbox_to_anchor=(1.05, 1))

plt.tight_layout()
plt.show()
# Estimation de l'intervalle de confiance à 95%
alpha = 0.05
z = 1.96 # quantile de la loi normale centrée réduite
CI_inf = BE - z*PM/np.sqrt(M)
CI_sup = BE + z*PM/np.sqrt(M)

# Affichage du Best Estimate et de la Prime de Risque
plt.bar(['BE', 'PM'], [BE, PM], yerr=[CI_sup-BE, 0], capsize=10)
plt.title('Best Estimate et Prime de Risque')
plt.ylabel('Valeur en euros')
plt.show()


fig, ax = plt.subplots(figsize=(8, 6))

cumulative_S = np.cumsum(S)
ax.plot(range(1, M+1), cumulative_S/np.arange(1, M+1))
ax.axhline(y=BE, color='r', label=f"Best Estimate : {BE:.2f}")
ax.set_xlabel("Nombre de simulations")
ax.set_ylabel("Valeur de S")
ax.set_title("Convergence de la méthode de Monte Carlo")
ax.legend()

plt.tight_layout()
plt.show()
fig, ax = plt.subplots(figsize=(8, 6))

ax.hist(S, bins=50, density=True, alpha=0.5)
ax.axvline(x=BE, color='r', label=f"Best Estimate : {BE:.2f}")
ax.set_xlabel("Valeur de S")
ax.set_ylabel("Densité de probabilité")
ax.set_title("Distribution des valeurs de S")
ax.legend()

plt.tight_layout()
plt.show()